In [1]:
import cv2
import pytesseract
import os

In [2]:
pytesseract.pytesseract.tesseract_cmd=r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [3]:
def detect_face(img):
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    face_cascade=cv2.CascadeClassifier(r"D:\Others\Projects\License Plate\haarcascade_frontalface_default.xml")
    face=face_cascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=4)
    if len(face)==0:
        return None,None
    x,y,w,h=face[0]
    return gray[x:x+w,y:y+h],face[0]

In [19]:
def prepare_training_data(data_folder_path):
    image_names=os.listdir(data_folder_path)
    faces=[]
    labels=[]
    for img_name in image_names:
        img_path=data_folder_path+'/'+img_name
        image=cv2.imread(img_path)
        cv2.imshow('training_image',image)
        cv2.waitKey(200)

        face,rect=detect_face(image)
        if face is not None:
            faces.append(face)
            labels.append(1)
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    return faces,labels

In [20]:
path=r"D:\Others\Projects\License Plate\set 2"
faces,labels=prepare_training_data(path)
print(labels)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [21]:
recognizer=cv2.face.LBPHFaceRecognizer_create()

In [22]:
import numpy as np

In [23]:
recognizer.train(faces,np.array(labels))

In [9]:
face_cascade=cv2.CascadeClassifier(r"D:\Others\Projects\License Plate\haarcascade_frontalface_default.xml")

In [17]:
def detect_plate(gray,test_img):
    plate_cas=cv2.CascadeClassifier(r"D:\Others\Projects\License Plate\haarcascade_russian_plate_number.xml")
    plate=plate_cas.detectMultiScale(test_img,minNeighbors=6,scaleFactor=1.2)
    for (x,y,w,h) in plate:
         cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),4)
         license_plate_coordinates=[]
         for i in x,y,w,h:
              license_plate_coordinates.append(i)
         license_plate_region = test_img[y:y+h, x:x+w]
         cv2.imshow('plate',license_plate_region)
         cv2.waitKey()
         cv2.destroyAllWindows()
         desired_width, desired_height = 200, 50
         preprocessed_license_plate = cv2.resize(license_plate_region, (desired_width, desired_height))
         license_plate_text = pytesseract.image_to_string(license_plate_region, config='--psm 9')
         print("License Plate Number:",license_plate_text)

In [18]:
test_img=cv2.imread(r"D:\Others\Projects\License Plate\bangalore_days_ver3.jpg")
gray=cv2.cvtColor(test_img,cv2.COLOR_BGR2GRAY)
faces=face_cascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=2)

for (x,y,w,h) in faces:
    face_roi = gray[y:y+h, x:x+w]
    label,confidence=recognizer.predict(face_roi)
    print(label)
    print(confidence)
    threshold=160
    if confidence > threshold: 
            cv2.rectangle(test_img, (x, y), (x+w, y+h), (0, 255, 0), 2)
            recognized_name = 'dulquer'
            cv2.putText(test_img, recognized_name, (x-14, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            cv2.imshow('predicted',test_img)
            cv2.waitKey()
            cv2.destroyAllWindows()
            detect_plate(gray,test_img)

1
157.23506829799646
1
172.78842201849216
License Plate Number: MMH 1403

